# Rekurentiniai neuroniniai tinklai

Ankstesniame modulyje aptarėme turtingas semantines teksto reprezentacijas. Naudota architektūra apima agreguotą žodžių prasmę sakinyje, tačiau ji neatsižvelgia į žodžių **tvarką**, nes agregavimo operacija, atliekama po įterpimų, pašalina šią informaciją iš pradinio teksto. Kadangi šie modeliai negali atspindėti žodžių tvarkos, jie negali spręsti sudėtingesnių ar dviprasmiškų užduočių, tokių kaip teksto generavimas ar klausimų atsakymas.

Norėdami užfiksuoti teksto sekos prasmę, naudosime neuroninių tinklų architektūrą, vadinamą **rekurentiniais neuroniniais tinklais** (RNN). Naudojant RNN, mes perduodame savo sakinį per tinklą po vieną žodį, o tinklas sukuria tam tikrą **būseną**, kurią vėliau perduodame tinklui kartu su kitu žodžiu.

![Paveikslėlis, rodantis rekurentinio neuroninio tinklo generavimo pavyzdį.](../../../../../lessons/5-NLP/16-RNN/images/rnn.png)

Turint įvesties žodžių seką $X_0,\dots,X_n$, RNN sukuria neuroninių tinklų blokų seką ir treniruoja šią seką nuo pradžios iki pabaigos naudodamas atgalinį sklidimą. Kiekvienas tinklo blokas kaip įvestį gauna porą $(X_i,S_i)$ ir kaip rezultatą sukuria $S_{i+1}$. Galutinė būsena $S_n$ arba išvestis $Y_n$ perduodama į linijinį klasifikatorių, kad būtų gautas rezultatas. Visi tinklo blokai dalijasi tais pačiais svoriais ir yra treniruojami nuo pradžios iki pabaigos per vieną atgalinio sklidimo etapą.

> Aukščiau pateiktame paveikslėlyje rekurentinis neuroninis tinklas parodytas išskleista forma (kairėje) ir kompaktiškesne rekurentine reprezentacija (dešinėje). Svarbu suprasti, kad visos RNN ląstelės turi tuos pačius **dalijamus svorius**.

Kadangi būsenos vektoriai $S_0,\dots,S_n$ perduodami per tinklą, RNN gali išmokti sekos priklausomybes tarp žodžių. Pavyzdžiui, kai žodis *ne* pasirodo kažkur sekoje, tinklas gali išmokti paneigti tam tikrus elementus būsenos vektoriuje.

Kiekvienoje RNN ląstelėje yra du svorio matricos: $W_H$ ir $W_I$, bei poslinkis $b$. Kiekviename RNN žingsnyje, turint įvestį $X_i$ ir įvesties būseną $S_i$, išvesties būsena apskaičiuojama kaip $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, kur $f$ yra aktyvavimo funkcija (dažnai $\tanh$).

> Tokiems uždaviniams kaip teksto generavimas (kurį aptarsime kitame skyriuje) ar mašininis vertimas, mes taip pat norime gauti tam tikrą išvesties reikšmę kiekviename RNN žingsnyje. Tokiu atveju yra dar viena matrica $W_O$, o išvestis apskaičiuojama kaip $Y_i=f(W_O\times S_i+b_O)$.

Pažiūrėkime, kaip rekurentiniai neuroniniai tinklai gali padėti klasifikuoti mūsų naujienų duomenų rinkinį.

> Smėliadėžės aplinkoje turime paleisti šią langelį, kad įsitikintume, jog reikalinga biblioteka yra įdiegta ir duomenys yra iš anksto užkrauti. Jei dirbate vietoje, galite praleisti šį langelį.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Kai treniruojami dideli modeliai, GPU atminties paskirstymas gali tapti problema. Taip pat gali tekti eksperimentuoti su skirtingais mini paketų dydžiais, kad duomenys tilptų į GPU atmintį, tačiau mokymas būtų pakankamai greitas. Jei vykdote šį kodą savo GPU kompiuteryje, galite eksperimentuoti su mini paketų dydžio koregavimu, kad paspartintumėte mokymą.

> **Note**: Kai kurios NVidia tvarkyklių versijos yra žinomos dėl to, kad po modelio mokymo neatlaisvina atminties. Šiame užrašų knygelėje vykdome kelis pavyzdžius, ir tai gali sukelti atminties išsekimą tam tikrose konfigūracijose, ypač jei atliekate savo eksperimentus toje pačioje užrašų knygelėje. Jei susiduriate su keistomis klaidomis pradėdami mokyti modelį, gali tekti iš naujo paleisti užrašų knygelės branduolį.


In [3]:
batch_size = 16
embed_size = 64

## Paprastas RNN klasifikatorius

Paprasto RNN atveju kiekvienas pasikartojantis vienetas yra paprastas linijinis tinklas, kuris priima įvesties vektorių ir būsenos vektorių, o tada sukuria naują būsenos vektorių. Keras bibliotekoje tai galima atvaizduoti naudojant `SimpleRNN` sluoksnį.

Nors galime tiesiogiai perduoti vieno karšto kodavimo (one-hot encoded) žetonus į RNN sluoksnį, tai nėra gera idėja dėl jų didelio dimensionalumo. Todėl naudosime įterpimo (embedding) sluoksnį, kad sumažintume žodžių vektorių dimensionalumą, po to RNN sluoksnį ir galiausiai `Dense` klasifikatorių.

> **Note**: Tais atvejais, kai dimensionalumas nėra toks didelis, pavyzdžiui, naudojant simbolių lygmens tokenizaciją, gali būti prasminga tiesiogiai perduoti vieno karšto kodavimo žetonus į RNN ląstelę.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Pastaba:** Čia naudojame netreniruotą įterpimo sluoksnį dėl paprastumo, tačiau geresniems rezultatams galime naudoti iš anksto apmokytą įterpimo sluoksnį, pasitelkiant Word2Vec, kaip aprašyta ankstesniame skyriuje. Būtų gera praktika pritaikyti šį kodą darbui su iš anksto apmokytais įterpimais.

Dabar apmokykime savo RNN. Apskritai, RNN yra gana sudėtinga treniruoti, nes kai RNN ląstelės yra išskleidžiamos pagal sekos ilgį, sluoksnių, dalyvaujančių atgaliniame sklidime, skaičius tampa labai didelis. Todėl turime pasirinkti mažesnį mokymosi greitį ir treniruoti tinklą su didesniu duomenų rinkiniu, kad gautume gerus rezultatus. Tai gali užtrukti gana ilgai, todėl rekomenduojama naudoti GPU.

Kad paspartintume procesą, RNN modelį treniruosime tik su naujienų antraštėmis, praleisdami aprašymą. Galite pabandyti treniruoti su aprašymu ir pažiūrėti, ar pavyks modelį apmokyti.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Pastaba**: tikslumas greičiausiai bus mažesnis, nes mokymui naudojami tik naujienų pavadinimai.


## Peržiūrint kintamųjų sekas

Atminkite, kad `TextVectorization` sluoksnis automatiškai užpildo kintamo ilgio sekas mini paketų viduje užpildymo ženklais. Pasirodo, kad šie ženklai taip pat dalyvauja mokyme ir gali apsunkinti modelio konvergenciją.

Yra keletas būdų, kaip sumažinti užpildymo kiekį. Vienas iš jų – pertvarkyti duomenų rinkinį pagal sekos ilgį ir grupuoti visas sekas pagal dydį. Tai galima padaryti naudojant funkciją `tf.data.experimental.bucket_by_sequence_length` (žr. [dokumentaciją](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Kitas būdas – naudoti **maskavimą**. Keras bibliotekoje kai kurie sluoksniai palaiko papildomą įvestį, kuri nurodo, kuriuos ženklus reikia atsižvelgti mokymo metu. Norėdami įtraukti maskavimą į mūsų modelį, galime arba pridėti atskirą `Masking` sluoksnį ([dokumentacija](https://keras.io/api/layers/core_layers/masking/)), arba nurodyti `mask_zero=True` parametrą mūsų `Embedding` sluoksnyje.

> **Note**: Šis mokymas užtruks apie 5 minutes, kad būtų baigtas vienas epochas visame duomenų rinkinyje. Jei pritrūksite kantrybės, galite bet kada nutraukti mokymą. Taip pat galite apriboti mokymui naudojamų duomenų kiekį, pridėdami `.take(...)` sąlygą po `ds_train` ir `ds_test` duomenų rinkinių.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Dabar, kai naudojame maskavimą, galime treniruoti modelį su visu antraščių ir aprašymų duomenų rinkiniu.

> **Pastaba**: Ar pastebėjote, kad naudojome vektorizatorių, ištreniruotą pagal naujienų antraštes, o ne visą straipsnio tekstą? Tai gali lemti, kad kai kurie žetonai bus ignoruojami, todėl geriau būtų pertreniruoti vektorizatorių. Tačiau tai gali turėti tik labai mažą poveikį, todėl dėl paprastumo laikysimės ankstesnio iš anksto ištreniruoto vektorizatoriaus.


## LSTM: Ilgalaikė trumpalaikė atmintis

Viena iš pagrindinių RNN problemų yra **nykstantys gradientai**. RNN tinklai gali būti gana ilgi, todėl jiems gali būti sunku perduoti gradientus atgal iki pat pirmojo tinklo sluoksnio atgalinio sklidimo metu. Kai taip nutinka, tinklas negali išmokti ryšių tarp tolimų žodžių. Vienas iš būdų išvengti šios problemos yra įvesti **aiškų būsenos valdymą** naudojant **vartus**. Dvi dažniausiai naudojamos architektūros, kurios naudoja vartus, yra **ilgalaikė trumpalaikė atmintis** (LSTM) ir **vartų relės vienetas** (GRU). Šiame skyriuje aptarsime LSTM.

![Paveikslėlis, rodantis ilgalaikės trumpalaikės atminties ląstelės pavyzdį](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM tinklas organizuotas panašiai kaip RNN, tačiau čia yra dvi būsenos, kurios perduodamos iš sluoksnio į sluoksnį: tikroji būsena $c$ ir paslėptas vektorius $h$. Kiekviename vienete paslėptas vektorius $h_{t-1}$ yra sujungiamas su įvestimi $x_t$, ir kartu jie kontroliuoja, kas vyksta su būsena $c_t$ ir išvestimi $h_{t}$ per **vartus**. Kiekvienas vartas turi sigmoidinę aktyvaciją (išvestis intervale $[0,1]$), kurią galima laikyti bitiniu masku, kai ji dauginama iš būsenos vektoriaus. LSTM turi šiuos vartus (nuo kairės į dešinę aukščiau esančiame paveikslėlyje):
* **užmaršumo vartai**, kurie nustato, kuriuos vektoriaus $c_{t-1}$ komponentus reikia pamiršti, o kuriuos perduoti toliau.
* **įvesties vartai**, kurie nustato, kiek informacijos iš įvesties vektoriaus ir ankstesnio paslėpto vektoriaus turėtų būti įtraukta į būsenos vektorių.
* **išvesties vartai**, kurie paima naują būsenos vektorių ir nusprendžia, kurie jo komponentai bus naudojami naujam paslėptam vektoriui $h_t$ sukurti.

Būsenos $c$ komponentus galima laikyti vėliavėlėmis, kurias galima įjungti arba išjungti. Pavyzdžiui, kai sekos metu sutinkame vardą *Alice*, galime spėti, kad tai moteris, ir pakelti vėliavėlę būsenoje, kuri nurodo, kad sakinyje yra moteriškos giminės daiktavardis. Kai toliau sutinkame žodžius *and Tom*, pakeliame vėliavėlę, kuri nurodo, kad dabar turime daugiskaitos daiktavardį. Taigi, manipuliuodami būsena, galime sekti sakinio gramatines savybes.

> **Note**: Štai puikus šaltinis, padedantis suprasti LSTM vidinę struktūrą: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) Christopher Olah.

Nors LSTM ląstelės vidinė struktūra gali atrodyti sudėtinga, Keras paslepia šią įgyvendinimą `LSTM` sluoksnyje, todėl vienintelis dalykas, kurį reikia padaryti aukščiau pateiktame pavyzdyje, yra pakeisti rekursinį sluoksnį:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Dvikryptės ir daugiasluoksnės RNN

Mūsų ankstesniuose pavyzdžiuose rekursiniai tinklai veikė nuo sekos pradžios iki pabaigos. Tai mums atrodo natūralu, nes atitinka kryptį, kuria skaitome ar klausomės kalbos. Tačiau scenarijams, kuriems reikia atsitiktinės prieigos prie įvesties sekos, logiškiau vykdyti rekursinį skaičiavimą abiem kryptimis. RNN, leidžiančios skaičiavimus abiem kryptimis, vadinamos **dvikryptėmis** RNN, ir jos gali būti sukurtos apgaubiant rekursinį sluoksnį specialiu `Bidirectional` sluoksniu.

> **Note**: `Bidirectional` sluoksnis sukuria dvi sluoksnio kopijas ir nustato vienos iš jų `go_backwards` savybę į `True`, kad ji eitų priešinga kryptimi per seką.

Rekursiniai tinklai, tiek vienkryptės, tiek dvikryptės, fiksuoja sekos šablonus ir saugo juos būsenos vektoriuose arba grąžina juos kaip išvestį. Kaip ir konvoliuciniuose tinkluose, mes galime sukurti dar vieną rekursinį sluoksnį po pirmojo, kad užfiksuotume aukštesnio lygio šablonus, sudarytus iš žemesnio lygio šablonų, kuriuos ištraukė pirmasis sluoksnis. Tai veda mus prie **daugiasluoksnės RNN** sąvokos, kurią sudaro du ar daugiau rekursinių tinklų, kur ankstesnio sluoksnio išvestis perduodama kitam sluoksniui kaip įvestis.

![Vaizdas, rodantis daugiasluoksnę ilgalaikės-trumpalaikės atminties RNN](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Paveikslėlis iš [šio puikaus straipsnio](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) autoriaus Fernando López.*

Keras leidžia lengvai sukurti šiuos tinklus, nes tereikia pridėti daugiau rekursinių sluoksnių prie modelio. Visuose sluoksniuose, išskyrus paskutinį, reikia nurodyti parametrą `return_sequences=True`, nes mums reikia, kad sluoksnis grąžintų visas tarpines būsenas, o ne tik galutinę rekursinio skaičiavimo būseną.

Sukurkime dviejų sluoksnių dvikryptę LSTM mūsų klasifikavimo užduočiai.

> **Note** šis kodas vėl užtrunka gana ilgai, tačiau jis suteikia didžiausią tikslumą, kokį iki šiol matėme. Taigi galbūt verta palaukti ir pamatyti rezultatą.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN kitiems užduočių tipams

Iki šiol mes daugiausia dėmesio skyrėme RNN naudojimui tekstų sekų klasifikavimui. Tačiau jos gali atlikti ir daugybę kitų užduočių, tokių kaip teksto generavimas ar mašininis vertimas — šias užduotis aptarsime kitame skyriuje.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
